In [2]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

Import data and check for null values

In [3]:
dataraw = pd.read_csv('trainwaterpump.csv')
dataraw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

Convert all string names to lowercase in order to compare them to each other

In [4]:
   
try:
    dataraw['funder'] = dataraw['funder'].apply(lambda x: str(x).lower())
    dataraw['installer'] = dataraw['installer'].apply(lambda x: str(x).lower())
    dataraw['basin'] = dataraw['basin'].apply(lambda x: str(x).lower())
except KeyError:
    dataraw[i] = 'zero'

Functions used to clean data

In [5]:
def replace_names(x):
    names = pd.unique(x.ravel())
    namesnum = range(0,len(x))
    namesdict = zip(names, namesnum)
    namesdict = dict(namesdict)
    return namesdict

# def data_drop(x,df):
#     for each in x:
#         return df.drop(each, axis = 1)
    
# def replace_values(data, x):
#     for i in range(0,len(x)):
#         return data.replace({x[i]:replace_names(data[x])})
    

Map all string values to intergers 

In [6]:
data = dataraw.replace({'funder':replace_names(dataraw.funder)})
data = data.replace({'installer':replace_names(data.installer)})
data = data.replace({'basin':replace_names(data.basin)})
data = data.replace({'subvillage':replace_names(data.subvillage)})
data = data.replace({'extraction_type_class':replace_names(data.extraction_type_class)})
data = data.replace({'management':replace_names(data.management)})
data = data.replace({'management_group':replace_names(data.management_group)})
data = data.replace({'payment':replace_names(data.payment)})
data = data.replace({'water_quality':replace_names(data.water_quality)})
data = data.replace({'quality_group':replace_names(data.quality_group)})
data = data.replace({'quantity':replace_names(data.quantity)})
data = data.replace({'source_type':replace_names(data.source_type)})
data = data.replace({'source_class':replace_names(data.source_class)})
data = data.replace({'waterpoint_type':replace_names(data.waterpoint_type)})
data = data.replace({'scheme_management':replace_names(data.scheme_management)})
data = data.replace({'scheme_name':replace_names(data.scheme_name)})
data = data.replace({'extraction_type':replace_names(data.extraction_type)})
data = data.replace({'lga':replace_names(data.lga)})
data = data.replace({'ward':replace_names(data.ward)})

Drop redundant columns

In [7]:
data = data.drop("quantity_group",axis = 1)
data = data.drop("wpt_name",axis = 1)
data = data.drop("waterpoint_type_group",axis = 1)
data = data.drop("source",axis = 1)
data = data.drop("payment_type",axis = 1)
data = data.drop("region", axis=1)
data = data.drop('recorded_by', axis = 1)
data = data.drop('extraction_type_group', axis = 1)
data = data.drop('public_meeting', axis =1)
data = data.drop("permit", axis=1)

Convert dates into number of days since first recorded date

In [8]:
from pandas import date_range
s = date_range(data.date_recorded.min(),data.date_recorded.max())
days = []
for i in range(0,len(s.values)):
    days.append((str(s.values[i]).split('T')[0]))
daynum = range(0,len(days))
days = zip(days,daynum)
daysdict = dict(days)
data = data.replace({'date_recorded':daysdict})
data.date_recorded = data.date_recorded.replace('2013-12-03',4069)

In [9]:
data.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,num_private,basin,...,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source_type,source_class,waterpoint_type
0,69572,6000,3074,0,1390,0,34.938093,-9.856322,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8776,0,3797,1,1399,1,34.698766,-2.147466,0,1,...,0,1,0,1,0,0,1,1,1,0
2,34310,25,3788,2,686,2,37.460664,-3.821329,0,2,...,0,0,0,2,0,0,0,2,1,1
3,67743,0,3760,3,263,3,38.486161,-11.155298,0,3,...,1,0,0,1,0,0,2,3,0,1
4,19728,0,3195,4,0,4,31.130847,-1.825359,0,1,...,0,2,1,1,0,0,3,1,1,0


Import labels and map to integer values

In [10]:
labels = pd.read_csv('trainlabels.csv')
labels.status_group = labels.status_group.map({'functional':1,'non functional':0,'functional needs repair':2})

In [ ]:
# for i in range(0,len(data.public_meeting.values)):
#     if data.public_meeting[i] == False:
#         data.public_meeting[i] = 0
#     else:
#         data.public_meeting[i] = 1
        

In [17]:
scalar = StandardScaler()
dataN = scalar.fit_transform(data.ix[:,1:])

In [19]:
dataN = pd.DataFrame(dataN, columns = data.ix[:,1:].columns)

In [21]:
dataN.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,num_private,basin,subvillage,...,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source_type,source_class,waterpoint_type
0,1.895665,-1.141136,-0.542187,1.041252,-0.477415,0.131052,-1.408791,-0.038749,-1.505187,-1.112424,...,-0.929625,-0.522373,-0.340028,-1.272516,-0.326656,-0.33666,-0.730311,-1.350286,-0.538410,-0.926551
1,-0.105970,1.022152,-0.538108,1.054237,-0.473470,0.094610,1.207934,-0.038749,-1.091822,-1.112238,...,-0.929625,-0.079115,-0.340028,-0.726817,-0.326656,-0.33666,0.280253,-0.921252,1.765074,-0.926551
2,-0.097630,0.995223,-0.534029,0.025541,-0.469524,0.515158,0.639751,-0.038749,-0.678456,-1.112051,...,-0.929625,-0.522373,-0.340028,-0.181119,-0.326656,-0.33666,-0.730311,-0.492219,1.765074,-0.066287
3,-0.105970,0.911444,-0.529950,-0.584751,-0.465578,0.671308,-1.849720,-0.038749,-0.265090,-1.111865,...,-0.179561,-0.522373,-0.340028,-0.726817,-0.326656,-0.33666,1.290816,-0.063185,-0.538410,-0.066287
4,-0.105970,-0.779092,-0.525871,-0.964200,-0.461632,-0.448669,1.317271,-0.038749,-1.091822,-1.111678,...,-0.929625,0.364144,0.940980,-0.726817,-0.326656,-0.33666,2.301380,-0.921252,1.765074,-0.926551


In [22]:
X = dataN
y = labels.ix[:,1:]

In [ ]:
model = SVC(kernel='linear',C=1).fit(X,y)